In [37]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

In [38]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie 2020')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [39]:
# USER INPUT CELL

# DV table parameters
DATABASE = 'Bernie 2020'
# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'unique_id' 
# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_cherdeman.contactibility_outcome'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = ['pickup_outcome']


# Modeling frame table parameters
# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_cherdeman.contactibility_modeling_frame'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['jsonid','state_code','census_block_group_2010', 'person_id', 'voter_phone']

# Output table parameters
# Schema to contain prediction tables
SCHEMA = 'bernie_cherdeman'
# String that will be concatenated in front of the output table's name
PREFIX = 'getthru'

# Sampling parameters
# Non-response training data
    # True: automatically select people not in DV_TABLE at random from Phoenix (assumes person_id is PRIMARY_KEY)
    # False: automatically select people where the DV equals 0 from the DV_TABLE
SAMPLE_FROM_PHOENIX = False
# Number of non-response classes per target class (default is 2) 
CLASS_BALANCE = 2
# Maximum number of targets to randomly sample from DV_TABLE
MAX_TARGET_COUNT = 40000

In [40]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [41]:
# Counts of positive classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)


In [42]:
sql_count_targets

[['pickup_outcome'], ['10657']]

In [43]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
# sample_share = []
# for i in range(len(DV_LIST)):
#     if int(sql_count_targets[1][i]) > MAX_TARGET_COUNT:
#         sql_count_targets[1][i] = MAX_TARGET_COUNT
#     u = round(int(sql_count_targets[1][i])*CLASS_BALANCE)
#     sample_share.append(u)      

In [44]:
# for i in range(len(DV_LIST)):
#     dv_item = DV_LIST[i]
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''


In [45]:
# Create training tables
# for i in range(len(DV_LIST)):
#     if (int(sql_count_targets[1][i])*3) <= 1000:
#         feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
#     if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
#         feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
#     if (int(sql_count_targets[1][i])*3) > 2000:
#         feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
#     dv_item = DV_LIST[i]
#     print(dv_item)
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''
#     training_sql = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_training_{i} CASCADE;
#     CREATE TABLE {SCHEMA}.{PREFIX}_training_{i} AS 
#     (select * from (
#     (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 1 order by random() limit 30000) 
#     union all 
#     {zero_sample})
#     inner join
#     (select {PRIMARY_KEY}, {feature_select} from {MODELING_FRAME}) using({PRIMARY_KEY}));"""
#     create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
#     create_training_sql.result().state
    
    

In [78]:
# Train models
train_list = []
model_list = []

for index, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    name = f"""{dv}_{datestamp}"""
    model = ModelPipeline(model='sparse_logistic',
                          dependent_variable=dv,
                          primary_key=PRIMARY_KEY,
                          excluded_columns=EXCLUSION_COLUMNS,
                          calibration='sigmoid',
                          model_name=name,
                          memory_requested=15000#,
                          #disk_requested=5
                         )
    
    where_string = '{} is not null'.format(dv)
    # Use 
    train = model.train(table_name=f"""bernie_cherdeman.contactibility_training""", 
                        database_name=DATABASE,
                        sql_where=where_string#,
                        #fit_params={'sample_weight': WEIGHT_VAR}
                       )
    
    model_list.append(model)
    train_list.append(train)    


TRAINING >>> pickup_outcome


In [79]:
#train_list[0].metadata

In [80]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

    
model_output, train_output = zip(*((m, t) for m, t in zip(model_output, train_output) if t in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success


In [81]:
# Generate validation metrics
metrics_list = []

for b in train_output:
    metric = {'job_id':b.job_id,
              'run_id':b.run_id,
              'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
              'model': b.metadata['model']['model'],
              'time_of_train_run': b.metadata['run']['time_of_run'],
              'n_rows': b.metadata['data']['n_rows'],
              'n_features': b.metadata['data']['n_cols'],
              'auc': b.metadata['metrics']['roc_auc'],
              'deciles': b.metadata['metrics']['deciles'],
              'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
              'accuracy': b.metadata['metrics']['accuracy'],
              'p_correct': b.metadata['metrics']['p_correct'],
              'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
              'feature_list':b.metadata['model']['parameters']['relvars']
             }
    metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true','feature_list'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true,feature_list
run_id,,,,,,,,,,,,,
204729837,59090702,pickup_outcome,sparse_logistic,2020-01-26T23:32:34Z,33576,524,0.786929,"[0.044967242406194166, 0.07655644921060471, 0....","[[19236, 2758], [6406, 5176]]",0.727067,"[0.8746021642266073, 0.4469003626316698]","[0.6550512270669526, 0.34494877293304743]","[phone_rank_1, phone_rank_2, phone_rank_3, pho..."


In [ ]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.{PREFIX}_validation_{datestamp}', 
                                                 existing_table_rows='drop')


In [22]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=10)
scores.result()


spoke_support_1box
spoke_persuasion_1plus
spoke_persuasion_1minus


{'container_id': 53702187,
 'error': None,
 'finished_at': '2019-12-22T01:57:04.000Z',
 'id': 193189502,
 'is_cancel_requested': False,
 'started_at': '2019-12-22T00:15:19.000Z',
 'state': 'succeeded'}

In [25]:
# Generate SQL for final output table and drop intermediary tables
input_train_list = []
output_score_list = []
for i in range(len(DV_LIST)):
    input_train = f"{SCHEMA}.{PREFIX}_training_{i}"
    input_train_list.append(input_train)
    output_score = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    output_score_list.append(output_score)

drop_input_train_sql = "\n".join(["drop table if exists {tbl};".format(tbl=v) for v in input_train_list])
drop_output_score_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in output_score_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(output_score_list) > 1:
    for i in output_score_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [27]:
output_table_sql = f"""
set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(output_score_list[0]) + ''.join(join_table) +');'  


In [28]:
print(output_table_sql)


set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS bernie_nmarchio2.spoke_output_20191221;
CREATE TABLE bernie_nmarchio2.spoke_output_20191221
  DISTSTYLE KEY
  DISTKEY (person_id)
  SORTKEY (person_id)
  AS (select person_id 
,spoke_support_1box_1 as spoke_support_1box
,spoke_persuasion_1plus_1 as spoke_persuasion_1plus
,spoke_persuasion_1minus_1 as spoke_persuasion_1minus
,NTILE(100) OVER (ORDER BY spoke_support_1box_1) AS spoke_support_1box_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1plus_1) AS spoke_persuasion_1plus_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1minus_1) AS spoke_persuasion_1minus_100 from bernie_nmarchio2.spoke_spoke_support_1box_20191221 left join bernie_nmarchio2.spoke_spoke_persuasion_1plus_20191221 using(person_id)  left join bernie_nmarchio2.spoke_spoke_persuasion_1minus_20191221 using(person_id) );


In [29]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


'succeeded'

In [30]:
# Drop intermediary tables
drop_input_train_query = civis.io.query_civis(sql=drop_input_train_sql, database=DATABASE)
drop_input_train_query.result().state

drop_output_score_query = civis.io.query_civis(sql=drop_output_score_sql, database=DATABASE)
drop_output_score_query.result().state

'succeeded'

In [31]:
print(drop_input_train_sql)
print(drop_output_score_sql)

drop table if exists bernie_nmarchio2.spoke_training_0;
drop table if exists bernie_nmarchio2.spoke_training_1;
drop table if exists bernie_nmarchio2.spoke_training_2;
drop table if exists bernie_nmarchio2.spoke_spoke_support_1box_20191221;
drop table if exists bernie_nmarchio2.spoke_spoke_persuasion_1plus_20191221;
drop table if exists bernie_nmarchio2.spoke_spoke_persuasion_1minus_20191221;


In [32]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_validation_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

'succeeded'

In [33]:
print(grant_statement)


GRANT ALL ON SCHEMA bernie_nmarchio2 TO GROUP bernie_data;
GRANT SELECT ON bernie_nmarchio2.spoke_output_20191221 TO GROUP bernie_data;
GRANT SELECT ON bernie_nmarchio2.spoke_validation_20191221 TO GROUP bernie_data;

